In [1]:
import json
import pandas as pd
with open('West Shetland PGDS - simple line - plan 4.json', "r") as json_file:
#     df = pd.read_json(json_file)
    f=json_file.read()
    data = json.loads(f)
#     df = pd.read_json(data[0])
len(data)

2

In [13]:
#level 1
for index, i in enumerate(data):
    print(index, i["name"])
    level = index
#     name=i["name"]
    l=0
    while "childern" in i:
        l+=1
        path="children"*l
        name = i["name"]

0 CAPEX West Shetland PGDS
1 OPEX West Shetland PGDS


In [14]:
(data[0]['children'][0]['children'][0]['children'][0]['children'][0]['children'][0])

{'name': None,
 'purpose': 'Procurement',
 'path': '/capex/simple line/OC-plet/OGWMPSLPI_OC-plet_001/Generic/null',
 'properties': ['Weight: 30,000 kg'],
 'min': None,
 'max': None,
 'amount': {'value': 780000, 'unit': '$'},
 'children': []}

In [15]:
def flatten_json(nested_json):
    """
        Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = []

    def flatten(x, name=''):
#         _out=[]
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
#                 _out.append(flatten(x[a], a))
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a)
#                 flatten(a, name + str(i) + '_')
                i += 1
        else:
            print(name)
            out[name[:-1]] = x
#             out.append({name: x})

    flatten(nested_json)
    return out

In [16]:
name

'OPEX West Shetland PGDS'

In [17]:
data2= flatten_json(data)

name_


TypeError: list indices must be integers or slices, not str

In [6]:
data2

NameError: name 'data2' is not defined

In [7]:
from pandas.io.json import json_normalize
json_normalize(data)

,name,purpose,path,properties,amount,children,min.value,min.unit,max.value,max.unit
0,CAPEX West Shetland PGDS,None,/capex,[],None,"[{'name': 'simple line', 'purpose': None, 'pat...",2.803073e+09,$,2.835300e+09,$
1,OPEX West Shetland PGDS,None,/opex,[],None,"[{'name': '2037', 'purpose': None, 'path': '/o...",4.440752e+08,$,4.440752e+08,$


In [11]:
from itertools import chain, starmap

def flatten_json_iterative_solution(dictionary):
    """Flatten a nested json file"""

    def unpack(parent_key, parent_value):
        """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                temp1 = parent_key + '_' + key
                yield temp1, value
        elif isinstance(parent_value, list):
            i = 0 
            for value in parent_value:
                temp2 = parent_key + '_'+str(i) 
                i += 1
                yield temp2, value
        else:
            yield parent_key, parent_value    

            
    # Keep iterating until the termination condition is satisfied
    while True:
        # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
        dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
        # Terminate condition: not any value in the json file is dictionary or list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

In [12]:
data3=flatten_json_iterative_solution(data)

AttributeError: 'list' object has no attribute 'items'

In [38]:
import json
# f = open("file", 'r')
global leafArray
leafArray = []


def parseTree(obj):
    if len(obj["children"]) == 0:
        leafArray.append(obj)
    else:
        for child in obj["children"]:
            parseTree(child)
for line in data:
    parseTree(line) 


In [39]:
df=json_normalize(leafArray)

In [40]:
df.to_excel("costtest.xlsx")

In [3]:
# This programm will create paths from root nodes to leafnodes along with values from any json file or structure.

import json
import pprint


json_data = open('West Shetland PGDS - simple line - plan 4.json', 'r').read()
json_dict = json.loads(json_data)

stack = []
final_dict = {}

def do_walk(datadict):

    if isinstance(datadict, dict):
        for key, value in datadict.items():
            stack.append(key)
            #print("/".join(stack))
            if isinstance(value, dict) and len(value.keys()) == 0:
                final_dict["/".join(stack)] = "EMPTY_DICT"
            if isinstance(value, list) and len(value) == 0:
                final_dict["/".join(stack)] = 'EMPTY_LIST'
            if isinstance(value, dict):
                do_walk(value)
            if isinstance(value, list):
                do_walk(value)
            if isinstance(value, str):
                final_dict["/".join(stack)] = value
            stack.pop()

    if isinstance(datadict, list):
        n = 0
        for key in datadict:
            stack.append(str(n))
            n = n + 1
            if isinstance(key, dict):
                do_walk(key)
            if isinstance(key, list):
                do_walk(key)
            if isinstance(key, str):
                final_dict["/".join(stack)] = key
            stack.pop()


do_walk(json_dict)
pprint.pprint(final_dict)

{'0/children/0/children/0/children/0/children/0/amount/unit': '$',
 '0/children/0/children/0/children/0/children/0/children/0/amount/unit': '$',
 '0/children/0/children/0/children/0/children/0/children/0/children': 'EMPTY_LIST',
 '0/children/0/children/0/children/0/children/0/children/0/path': '/capex/simple '
                                                                  'line/OC-plet/OGWMPSLPI_OC-plet_001/Generic/null',
 '0/children/0/children/0/children/0/children/0/children/0/properties/0': 'Weight: '
                                                                          '30,000 '
                                                                          'kg',
 '0/children/0/children/0/children/0/children/0/children/0/purpose': 'Procurement',
 '0/children/0/children/0/children/0/children/0/name': 'Generic',
 '0/children/0/children/0/children/0/children/0/path': '/capex/simple '
                                                       'line/OC-plet/OGWMPSLPI_OC-plet_001/Generic',

                                                                  'Gas/_CS '
                                                                  'line '
                                                                  'Gas_001/Components/Bare '
                                                                  'pipe',
 '0/children/0/children/2/children/0/children/0/children/0/properties': 'EMPTY_LIST',
 '0/children/0/children/2/children/0/children/0/children/1/children': 'EMPTY_LIST',
 '0/children/0/children/2/children/0/children/0/children/1/max/unit': '$',
 '0/children/0/children/2/children/0/children/0/children/1/min/unit': '$',
 '0/children/0/children/2/children/0/children/0/children/1/name': 'FJC',
 '0/children/0/children/2/children/0/children/0/children/1/path': '/capex/simple '
                                                                  'line/CS '
                                                                  'line '
                                                       

 '0/children/0/children/2/children/0/children/0/children/4/children/1/children/1/purpose': 'Transport',
 '0/children/0/children/2/children/0/children/0/children/4/children/1/name': 'Generic',
 '0/children/0/children/2/children/0/children/0/children/4/children/1/path': '/capex/simple '
                                                                             'line/CS '
                                                                             'line '
                                                                             'Gas/_CS '
                                                                             'line '
                                                                             'Gas_001/Components/Bare '
                                                                             'pipe/Generic',
 '0/children/0/children/2/children/0/children/0/children/4/children/1/properties': 'EMPTY_LIST',
 '0/children/0/children/2/children/0/children/0/children/4/max/unit': '$',

                                                                             '4/csv/Precom-dewatering/Mobilisation/Vessel '
                                                                             'owner '
                                                                             '1/null',
 '0/children/1/children/4/children/2/children/0/children/0/children/0/properties': 'EMPTY_LIST',
 '0/children/1/children/4/children/2/children/0/children/0/children/0/purpose': 'Total '
                                                                                'cost '
                                                                                'vessel '
                                                                                '+ '
                                                                                'spread',
 '0/children/1/children/4/children/2/children/0/children/0/name': 'Vessel '
                                                                  'owner 1',
 '0/childre

 '1/children/0/children/0/children/2/children/1/children/2/path': '/opex/2062/OC-plet/OC-plet '
                                                                  '1/Components/Vertical '
                                                                  'connector',
 '1/children/0/children/0/children/2/children/1/children/2/properties': 'EMPTY_LIST',
 '1/children/0/children/0/children/2/children/1/children/3/children': 'EMPTY_LIST',
 '1/children/0/children/0/children/2/children/1/children/3/max/unit': '$',
 '1/children/0/children/0/children/2/children/1/children/3/min/unit': '$',
 '1/children/0/children/0/children/2/children/1/children/3/name': 'Pressure '
                                                                  'cap',
 '1/children/0/children/0/children/2/children/1/children/3/path': '/opex/2062/OC-plet/OC-plet '
                                                                  '1/Components/Pressure '
                                                                  'cap',
 '1

...

 '1/children/3/children/0/children/0/children/1/children/2/min/unit': '$',
 '1/children/3/children/0/children/0/children/1/children/2/name': 'Bend',
 '1/children/3/children/0/children/0/children/1/children/2/path': '/opex/2062/OC-plet/OGWMPSLPI_OC-plet_001/Components/Bend',
 '1/children/3/children/0/children/0/children/1/children/2/properties': 'EMPTY_LIST',
 '1/children/3/children/0/children/0/children/1/children/3/children': 'EMPTY_LIST',
 '1/children/3/children/0/children/0/children/1/children/3/max/unit': '$',
 '1/children/3/children/0/children/0/children/1/children/3/min/unit': '$',
 '1/children/3/children/0/children/0/children/1/children/3/name': 'Mudmat',
 '1/children/3/children/0/children/0/children/1/children/3/path': '/opex/2062/OC-plet/OGWMPSLPI_OC-plet_001/Components/Mudmat',
 '1/children/3/children/0/children/0/children/1/children/3/properties': 'EMPTY_LIST',
 '1/children/3/children/0/children/0/children/1/children/4/children': 'EMPTY_LIST',
 '1/children/3/children/0/child

 '1/children/4/children/1/children/0/children/0/amount/unit': '$',
 '1/children/4/children/1/children/0/children/0/children/0/amount/unit': '$',
 '1/children/4/children/1/children/0/children/0/children/0/children': 'EMPTY_LIST',
 '1/children/4/children/1/children/0/children/0/children/0/path': '/opex/2062/CS '
                                                                  'line/_CS '
                                                                  'line_001/Accruals/null',
 '1/children/4/children/1/children/0/children/0/children/0/properties/0': 'Length: '
                                                                          '181,016.96 '
                                                                          'm',
 '1/children/4/children/1/children/0/children/0/children/0/purpose': 'field '
                                                                     'abandonment',
 '1/children/4/children/1/children/0/children/0/name': 'Accruals',
 '1/children/4/children/1/children/0/

 '1/children/6/children/0/children/1/children/1/children/0/children': 'EMPTY_LIST',
 '1/children/6/children/0/children/1/children/1/children/0/max/unit': '$',
 '1/children/6/children/0/children/1/children/1/children/0/min/unit': '$',
 '1/children/6/children/0/children/1/children/1/children/0/name': 'Yoke',
 '1/children/6/children/0/children/1/children/1/children/0/path': '/opex/2062/OC-plet/OGWMPSLPI_OC-plet_002/Components/Yoke',
 '1/children/6/children/0/children/1/children/1/children/0/properties': 'EMPTY_LIST',
 '1/children/6/children/0/children/1/children/1/children/1/children': 'EMPTY_LIST',
 '1/children/6/children/0/children/1/children/1/children/1/max/unit': '$',
 '1/children/6/children/0/children/1/children/1/children/1/min/unit': '$',
 '1/children/6/children/0/children/1/children/1/children/1/name': 'Vertical '
                                                                  'connector',
 '1/children/6/children/0/children/1/children/1/children/1/path': '/opex/2062/OC-plet/OGW

 '1/children/7/children/2/children/0/children/1/children/2/purpose': 'personnel '
                                                                     'logistics',
 '1/children/7/children/2/children/0/children/1/children/3/amount/unit': '$',
 '1/children/7/children/2/children/0/children/1/children/3/children': 'EMPTY_LIST',
 '1/children/7/children/2/children/0/children/1/children/3/path': '/opex/2062/OC-fpso/OC-fpso '
                                                                  '1/Operations/null',
 '1/children/7/children/2/children/0/children/1/children/3/properties/0': 'Weight: '
                                                                          '300,000,000 '
                                                                          'kg',
 '1/children/7/children/2/children/0/children/1/children/3/purpose': 'maintenance',
 '1/children/7/children/2/children/0/children/1/children/4/amount/unit': '$',
 '1/children/7/children/2/children/0/children/1/children/4/children': 'EMPT

 '1/children/9/children/1/children/0/children/0/properties': 'EMPTY_LIST',
 '1/children/9/children/1/children/0/children/1/amount/unit': '$',
 '1/children/9/children/1/children/0/children/1/children/0/amount/unit': '$',
 '1/children/9/children/1/children/0/children/1/children/0/children': 'EMPTY_LIST',
 '1/children/9/children/1/children/0/children/1/children/0/path': '/opex/2062/CS '
                                                                  'line/_CS '
                                                                  'line_001/Operations/null',
 '1/children/9/children/1/children/0/children/1/children/0/properties/0': 'Length: '
                                                                          '181,016.96 '
                                                                          'm',
 '1/children/9/children/1/children/0/children/1/children/0/purpose': 'wall '
                                                                     'thickness '
                              